### PostgreSQL ↔ Elasticsearch

Il existe plusieurs façons de synchroniser PostgreSQL avec Elasticsearch :

|Méthode|Avantages|Inconvénients|
|:---|:---|:---|
|Triggers + pg_notify|Réactif et efficace|Besoin d'un service externe pour traiter les notifications|
|Python (Elasticsearch-Py)|Contrôle total sur la synchronisation|Code personnalisé à écrire|
|Logstash (JDBC Input Plugin)|Configuration simple|Temps de latence entre les mises à jour|
|Debezium (Change Data Capture - CDC)|Réplication en temps réel|Nécessite Kafka ou Redis|

### Triggers PostgreSQL + pg_notify

L'idée ici est que chaque modification dans PostgreSQL envoie un événement à un système externe (ex: un script Python qui écoute PostgreSQL et met à jour Elasticsearch).

- Création d’un trigger pour notifier les mises à jour

```sql
CREATE OR REPLACE FUNCTION notify_elasticsearch()
RETURNS TRIGGER AS $$
DECLARE
    json_data TEXT;
BEGIN
    json_data := json_build_object(
        'action', TG_OP,  -- INSERT, UPDATE, DELETE
        'id', NEW.id,
        'nom', NEW.nom,
        'description', NEW.description,
        'prix', NEW.prix
    )::TEXT;

    PERFORM pg_notify('elasticsearch_sync', json_data);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

- Associer la fonction à un trigger

```sql
CREATE TRIGGER trigger_elasticsearch
AFTER INSERT OR UPDATE ON produits
FOR EACH ROW EXECUTE FUNCTION notify_elasticsearch();
```

= Maintenant, à chaque modification, PostgreSQL enverra un événement pg_notify avec les données du produit.